# 퓨처스리그 ( 항공 지연 분석)

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
import matplotlib.pyplot as  plt
import time

## 데이터 업로드

In [2]:
AFSNT = pd.read_csv("AFSNT.csv", engine='python')          # train table

AFSNT_DLY = pd.read_csv("AFSNT_DLY.csv", engine='python')   # Target table

SFSNT = pd.read_csv("SFSNT.csv", engine='python')       # 비행기의 요일별 출항유무

## 데이터 형식 살펴보기

In [5]:
AFSNT.info()

SFSNT.info()

print(AFSNT.describe())

print(SFSNT.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987709 entries, 0 to 987708
Data columns (total 17 columns):
SDT_YY    987709 non-null int64
SDT_MM    987709 non-null int64
SDT_DD    987709 non-null int64
SDT_DY    987709 non-null object
ARP       987709 non-null object
ODP       987709 non-null object
FLO       987709 non-null object
FLT       987709 non-null object
REG       987251 non-null object
AOD       987709 non-null object
IRR       987709 non-null object
STT       987709 non-null object
ATT       979461 non-null object
DLY       987709 non-null object
DRR       119917 non-null object
CNL       987709 non-null object
CNR       3018 non-null object
dtypes: int64(3), object(14)
memory usage: 128.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171 entries, 0 to 1170
Data columns (total 17 columns):
SSC    1171 non-null object
FLT    1171 non-null object
ORG    1171 non-null object
DES    1171 non-null object
STD    1153 non-null object
STA    1152 non-null object
FLO  

## 분석에 필요한 칼럼 남기기
### 지연 이유가 A(기상관련 지연 이유)로 시작하는 지연을 제외한 나머지 지연을 합치는 작업.
### 지연과 결항도 합쳐서 하나의 칼럼으로 생성

In [44]:
Test = AFSNT.copy()

Test.drop(["SDT_YY", "SDT_DD", "SDT_MM", "REG", "IRR", "STT", "ATT"], axis = 1, inplace = True)

b =((Test["CNR"] != 'A05') & (Test["CNR"] != 'A01') & (Test["CNR"] != 'A02') &(Test["CNR"] != 'A07') & (Test["CNR"] != 'A03') & (Test["CNR"] != 'A10') & (Test["CNR"] != 'A99')& (Test["CNR"] != 'A09') & (Test["CNR"] != 'A04'))

a = ((Test["DRR"] != 'A05') & (Test["DRR"] != 'A01') & (Test["DRR"] != 'A02') & (Test["DRR"] != 'A07') & (Test["DRR"] != 'A03') & (Test["DRR"] != 'A10') & (Test["DRR"] != 'A99') & (Test["DRR"] != 'A09') & (Test["DRR"] != 'A04') & (Test["DRR"] != 'A06'))

Test["DLYCNL"] = 0

Test.loc[((Test["DLY"] == "Y") & a) | ((Test["CNL"] == "Y") & b) , ["DLYCNL"] ] = 1

Test.drop(["DRR", "DLY", "CNL", "CNR"], axis=1, inplace=True)

In [45]:
Test.head()

,SDT_DY,ARP,ODP,FLO,FLT,AOD,DLYCNL
0,일,ARP1,ARP3,A,A1901,D,0
1,일,ARP1,ARP3,A,A1905,D,0
2,일,ARP1,ARP3,L,L1751,D,0
3,일,ARP1,ARP3,F,F1201,D,0
4,일,ARP3,ARP1,A,A1900,D,0


### FLT Columns 과 Target Columns 을 제외한 나머지 Columns 원핫인코딩 및 float 형으로 바꿔주기
### FLT는 value 값이 많아 그냥 수치화로 했습니다.
### 분석할 최종 Test File 형식에 맞게 전처리하기.

In [46]:
Test_DLY = AFSNT_DLY.copy()

Test_DLY.drop(["SDT_YY", "SDT_MM", "SDT_DD", "STT", "DLY", "DLY_RATE"], axis=1, inplace = True)

Test_DLY["DLYCNL"] = 0

Test_concat = pd.concat([Test, Test_DLY], sort=False)

In [58]:
Test

,SDT_DY,ARP,ODP,FLO,FLT,AOD,DLYCNL
0,일,ARP1,ARP3,A,A1901,D,0
1,일,ARP1,ARP3,A,A1905,D,0
2,일,ARP1,ARP3,L,L1751,D,0
3,일,ARP1,ARP3,F,F1201,D,0
4,일,ARP3,ARP1,A,A1900,D,0
5,일,ARP1,ARP3,H,H1101,D,0
6,일,ARP1,ARP3,I,I1301,D,0
7,일,ARP1,ARP3,A,A1907,D,1
8,일,ARP1,ARP3,I,I1303,D,0
9,일,ARP1,ARP3,J,J1204,D,0


In [50]:
le = LabelEncoder()

X = Test_concat.iloc[:, :].values

X[:, 4] = le.fit_transform(X[:, 4])

Test_concat["FLT"] = X[:, 4]

Test1 = Test_concat.drop("FLT", axis =1)

Test_dummie = pd.get_dummies(Test1.iloc[:, :-1])

Test_dummie = pd.concat([Test_dummie, Test_concat["FLT"]], axis = 1)

In [59]:
Test_dummie[987709:].head()

,SDT_DY_금,SDT_DY_목,SDT_DY_수,SDT_DY_월,SDT_DY_일,SDT_DY_토,SDT_DY_화,ARP_ARP1,ARP_ARP10,ARP_ARP11,...,FLO_G,FLO_H,FLO_I,FLO_J,FLO_K,FLO_L,FLO_M,AOD_A,AOD_D,FLT
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1450
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1450
2,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1468
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1468
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1517


## Sklearn 을 활용한 분석 
### Linear 방식을 사용한 분석은 정확도가 너무 낮아 사용하지 않았습니다.

In [13]:
x_train, x_test, y_train, y_test = train_test_split(Test_dummie[:987709], Test["DLYCNL"], random_state=0)

## KNeighborsClassifier 이용한 분석

In [ ]:
knn = KNeighborsClassifier()
x = [ x for x in range(1, 100, 3)]
neighbors = {'n_neighbors' : x }
grid_searchKNN = GridSearchCV(knn, neighbors)
grid_searchKNN.fit(x_train, y_train)

In [29]:
print(grid_searchKNN.score(x_test, y_test))
print(grid_searchKNN.best_params_)
print(grid_searchKNN.best_score_)

0.8765470096546362
{'n_neighbors': 88}
0.8769001904746477


## DecisionTree를 이용한 분석 

In [ ]:
Tree = DecisionTreeClassifier(random_state = 0)
x = [ x for x in range(1, 31,3)]
depth = {'max_depth' : x }
grid_searchTree = GridSearchCV(Tree, depth)
grid_searchTree.fit(x_train, y_train)

In [33]:
print(grid_searchTree.score(x_test, y_test))
print(grid_searchTree.best_params_)
print(grid_searchTree.best_score_)

0.8762351778656127
{'max_depth': 10}
0.8767152505261339


##  RandomForestClassifier 를 이용한 분석

In [34]:
Rf = RandomForestClassifier(random_state = 0)
x = [ x for x in range(1, 502, 100)]
estimators = {'n_estimators' : x }
grid_searchRf = GridSearchCV(Rf, estimators)
grid_searchRf.fit(x_train, y_train)
print(grid_searchRf.score(x_test, y_test))
print(grid_searchRf.best_params_)
print(grid_searchRf.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8763485712434393
{'n_estimators': 501}
0.8761860792865908


## GradientBoostingClassifier 를 이용한 분석

In [35]:
Gbrt = GradientBoostingClassifier(random_state = 0)
x = [ x for x in range(1, 22, 3)]
depth = {'max_depth' : x }
grid_searchGbrt = GridSearchCV(Gbrt, depth)
grid_searchGbrt.fit(x_train, y_train)
print(grid_searchGbrt.score(x_test, y_test))
print(grid_searchGbrt.best_params_)
print(grid_searchGbrt.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8765551091816238
{'max_depth': 10}
0.8770392329176909


## MLPClassifier 를 이용한 분석

In [37]:
MLP = MLPClassif ier(random_state = 0)
hidden = {'hidden_layer_sizes' : [[10, 10],  [50, 50],  [100,100]], 'alpha' : [0.0001, 0.01, 1], 'solver' : ['adam', 'lbfgs'] }
grid_searchMLP = GridSearchCV(MLP, hidden)
grid_searchMLP.fit(x_train, y_train)
print(grid_searchMLP.score(x_test, y_test))
print(grid_searchMLP.best_params_)
print(grid_searchMLP.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8758706991511696
{'alpha': 0.0001, 'hidden_layer_sizes': [10, 10], 'solver': 'adam'}
0.8764439152732049


## Perceptron 을 이용한 분석

In [36]:
rfP = Perceptron()
rfP.fit(x_train, y_train)
print("Perceptron")
print(rfP.score(x_train, y_train))
print(rfP.score(x_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Perceptron
0.8752006328456048
0.8745666753061622


## 모델을 활용한 정답지 추출하기

In [76]:
KNN_predict = grid_searchKNN.predict(Test_dummie[987709:])
Tree_predict = grid_searchTree.predict(Test_dummie[987709:])
RF_predict = grid_searchRf.predict(Test_dummie[987709:])
GBRT_predict = grid_searchGbrt.predict(Test_dummie[987709:])
MLP_predict = grid_searchMLP.predict(Test_dummie[987709:])
PE_predict = rfP.predict(Test_dummie[987709:])
Total_predict = KNN_predict + Tree_predict + RF_predict + GBRT_predict + MLP_predict + PE_predict

In [77]:
KNN_proba = grid_searchKNN.predict_proba(Test_dummie[987709:])
Tree_proba = grid_searchTree.predict_proba(Test_dummie[987709:])
RF_proba = grid_searchRf.predict_proba(Test_dummie[987709:])
GBRT_proba = grid_searchGbrt.predict_proba(Test_dummie[987709:])
MLP_proba = grid_searchMLP.predict_proba(Test_dummie[987709:])

In [95]:
KNN_proba = pd.DataFrame(KNN_proba)
Tree_proba = pd.DataFrame(Tree_proba)
RF_proba = pd.DataFrame(RF_proba)
GBRT_proba = pd.DataFrame(GBRT_proba)
MLP_proba = pd.DataFrame(MLP_proba)

In [96]:
KNN_proba1 = KNN_proba[1].map(lambda x = 1 : x == 1)
KNN_proba1 = KNN_proba1.map({False : 0 , True : 1})

Tree_proba1 = Tree_proba[1].map(lambda x = 1 : x == 1)
Tree_proba1 = Tree_proba1.map({False : 0 , True : 1})

RF_proba1 = RF_proba[1].map(lambda x = 1 : x > 0.99)
RF_proba1 = RF_proba1.map({False : 0 , True : 1})

GBRT_proba1 = GBRT_proba[1].map(lambda x = 1 : x > 0.46)
GBRT_proba1 = GBRT_proba1.map({False : 0 , True : 1})

MLP_proba1 = MLP_proba[1].map(lambda x = 1 : x > 0.4)
MLP_proba1 = MLP_proba1.map({False : 0 , True : 1})

In [97]:
Total_proba = KNN_proba1 + Tree_proba1 + RF_proba1 + GBRT_proba1 + MLP_proba1

In [100]:
Total_proba = Total_proba.map({0 : 0, 1 : 1.5, 2 : 3, 3 : 4.5, 4 : 6})

In [104]:
Total_predict = pd.DataFrame(Total_predict)

In [112]:
Total = Total_proba + Total_predict[0]

## 정답지 적용하여 CSV 파일 만들기

In [128]:
Total_DLY = AFSNT_DLY.copy()

In [137]:
Total.map(lambda x : x > 2).map({False : 'N', True : 'Y'})

0        N
1        N
2        N
3        N
4        N
5        N
6        N
7        N
8        N
9        N
10       N
11       N
12       N
13       N
14       N
15       N
16       N
17       N
18       N
19       N
20       N
21       N
22       N
23       N
24       N
25       N
26       N
27       N
28       N
29       N
        ..
16124    N
16125    N
16126    N
16127    N
16128    N
16129    N
16130    N
16131    N
16132    N
16133    N
16134    N
16135    N
16136    N
16137    N
16138    N
16139    N
16140    N
16141    N
16142    N
16143    N
16144    N
16145    N
16146    N
16147    N
16148    N
16149    N
16150    N
16151    N
16152    N
16153    N
Length: 16154, dtype: object

In [139]:
Total_DLY["DLY"] = Total.map(lambda x : x > 2).map({False : 'N', True : 'Y'})

In [140]:
Total.value_counts()

0.0    15934
1.0       67
4.5       45
2.0       35
3.5       27
5.5       15
1.5       13
2.5       13
9.0        3
3.0        2
dtype: int64

In [149]:
Total_DLY['DLY_RATE'] = Total.map({0 : 0, 1 : 0, 2 : 0, 4.5 : 4.5, 1.5 : 0, 3.5 : 3.5, 5.5 : 5.5, 2.5 : 2.5, 9 : 8, 3 : 3})*12

In [150]:
Total_DLY.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,FLT,AOD,STT,DLY,DLY_RATE
0,2019,9,16,월,ARP1,ARP3,L,L1702,A,9:05,N,0.0
1,2019,9,16,월,ARP3,ARP1,L,L1702,D,7:55,N,0.0
2,2019,9,16,월,ARP1,ARP3,L,L1720,A,14:40,N,0.0
3,2019,9,16,월,ARP3,ARP1,L,L1720,D,13:30,N,0.0
4,2019,9,16,월,ARP4,ARP3,L,L1808,A,20:10,N,0.0


In [151]:
Total_DLY.to_csv("정답지.csv", encoding='CP949')